In [6]:
import subprocess
import os
import jaccard.jaccard as jac
import binary_ast.compare_ast as bin_ast
from pathlib import Path

cwd = os.getcwd()
os.putenv("GHIDRA_HEADLESS", "C:\\Users\\wamuo\Documents\\Tools\\ghidra_10.1_PUBLIC\\support\\analyzeHeadless")

In [7]:
#Import all packages and at base #20004000 and run ghidra analysis to find functions
#Create a new folder for all analyzed binary using the version number
#For all binaries extract generated instructions for all functions and store in a file named info_graph.json under the version folder
# ghidra_folder = Path("./ghidra_mods")

if os.name == 'nt':
    p = subprocess.Popen(["powershell",".\ghidra_mods\ghidra_pipeline.ps1",cwd], stdout=subprocess.PIPE)
    p.communicate()
else:
    subprocess.call(['./ghidra_mods/ghidra_pipeline.sh', cwd])


In [8]:
subfolders = [ f.path for f in os.scandir("versions") if f.is_dir() ]
if os.name != 'nt':
    subfolders.reverse()
binary_pairs = [(subfolders[i - 1], subfolders[i]) for i in range(1, len(subfolders))]
print(binary_pairs)

[('versions\\otaApp-1_4_2_bin', 'versions\\otaApp-1_4_4_bin'), ('versions\\otaApp-1_4_4_bin', 'versions\\otaApp-1_5_3_bin'), ('versions\\otaApp-1_5_3_bin', 'versions\\otaApp-1_5_4_bin'), ('versions\\otaApp-1_5_4_bin', 'versions\\otaApp-1_5_7_bin')]


In [9]:
graph_file_name = "/info_graph.json"

possible_updates_by_binary_pair = {}
for binary_pair in binary_pairs:
    possible_updates = jac.calculate_jaccard(binary_pair[0]+ graph_file_name, binary_pair[1] + graph_file_name, binary_pair[1], 0.85)
    possible_updates_by_binary_pair[binary_pair] = possible_updates
    

In [ ]:
for binary_pair in possible_updates_by_binary_pair:
    binary_a = binary_pair[0].split(os.sep)[1].replace('_', '.')
    binary_b = binary_pair[1].split(os.sep)[1].replace('_', '.')
    func_pairs = possible_updates_by_binary_pair[binary_pair]

    for func_pair in func_pairs:
        print(func_pair)
#         if "0x2000b300" not in func_pair[1]:
#             continue
        
        func_a = func_pair[1][1]
        func_b = func_pair[1][0]
        
        funcA_ast_file = binary_pair[0] + os.sep + func_a + ".json"
        funcB_ast_file = binary_pair[1] + os.sep + func_b + ".json"

        if not os.path.exists(funcA_ast_file):
            if os.name == 'nt':
                p = subprocess.Popen(["powershell",".\ghidra_mods\win_ast.ps1",cwd, binary_a, func_a, binary_pair[0]], stdout=subprocess.PIPE)
                p_o, p_e = p.communicate()
            else:
                subprocess.call(['./ghidra_mods/ast.sh', cwd, binary_a, func_a, binary_pair[0]])
        
        if not os.path.exists(funcB_ast_file):
            if os.name == 'nt':
                p = subprocess.Popen(["powershell",".\ghidra_mods\win_ast.ps1", cwd, binary_b, func_b, binary_pair[1]], stdout=subprocess.PIPE)
                p.communicate()
            else:
                subprocess.call(['./ghidra_mods/ast.sh', cwd, binary_b, func_b, binary_pair[1]])
        print("\n\n\nNew Comparison \n", func_a, func_b)
        path = binary_pair[1] + os.sep + func_a +"_to_" + func_b 
        bin_ast.compare_ast(binary_pair[0], binary_pair[1], funcA_ast_file, funcB_ast_file, path)

([0.0, 0.2138687943262409, 0.875], ['0x20009032', '0x2000915e'])



New Comparison 
 0x2000915e 0x20009032
Tree size =  67
Tree size =  48
removes ['ram:20009186 o 52']
removing ram:20009186 o 52
['i v 602', 'i v 131', 'i v 591', 'i v 605', 'i v 615', 'i v 621', 'ram:20009186 o 52', 'i v 604', 'ram:20009184 o 48', 'ram:20009184 v 628', 'ram:20009186 o 216', 'ram:20009186 v 662', 'i v 589', 'ram:20009172 o 37', 'ram:20009172 v 627', 'ram:20009186 o 220', 'ram:20009186 v 677', 'i v 758', 'ram:2000919e o 213', 'ram:2000919e v 651', 'ram:200091a4 o 215', 'ram:200091a4 v 658']
removing i v 131
['i v 602', 'i v 131', 'i v 591', 'i v 605', 'i v 615', 'i v 621', 'i v 604', 'ram:20009184 o 48', 'ram:20009184 v 628', 'ram:20009186 o 216', 'ram:20009186 v 662', 'i v 589', 'ram:20009172 o 37', 'ram:20009172 v 627', 'ram:20009186 o 220', 'ram:20009186 v 677', 'i v 758', 'ram:2000919e o 213', 'ram:2000919e v 651', 'ram:200091a4 o 215', 'ram:200091a4 v 658']
removing i v 591
['i v 602', 'i v 591', 'i